# Backpropogation from Scratch 

#### Final Project for COMPSCIX404 - Data Structures and Algorithms
#### Nikhil Gopal 
#### August 2024

In this project, I will build the backpropogation algorithm, and show how it can be used to compute the gradient of the cost function when training a Neural Network. Neural Networks trained with backpropogation can be applied to text generation, computer vision, classification, facial recognition, audio generation, speech to text and many more applications.

I will start by explaining the algorithm and it's applicability to modern Deep Learning, with an overview of the mathematics behind it. Then, I will analyze the time and space complexity of the algorithm before presenting a code implementation.

Finally, we'll use the Pytorch Deep Learning Framework to show a real life example of how a Data Scientist or Deep Learning Engineer would train a Neural Network to classify handwritten digits from the popular MNSIT dataset. Pytorch automatically computes derivatives and gradients for you with the `loss.backwards()` method, but I will still show it so that you get an understanding of how this would be applied in the real world. 




## Intro and Motivation

Fundamentally, every problem in Machine Learning is an **optimization problem**. Optimization is the process of finding the maximum or minimum value of a function, such that it behaves optimally. In the case of Machine Learning, we seek to optimize the *loss function*, which describes the difference between the expected output of an ML model, and it's actual output. For example, if we were predicting human height and our model predicted 100 cm for a person who's true height was 105 cm, the loss would be $105-100=5$ cm. ML models have changable *parameters* whose optimal values can be learned from data such that the model's loss function is as close as possible to zero. 


Deep Learning is a subsect of Machine Learning that uses a specific type of Machine Learning model called a Multi-Layer Perceptron, also commonly referred to as Neural Networks. Neural Networks pass an input through different "layers" of the model, which all contribute to the model's final output. These models often have much higher numbers of learnable parameters than simpler models, but the computations corresponding to a single input must be performed sequentially and cannot be parallelized, since the input of a later layer must be the output of a previous layer:

**Input image depicting Neural Net here**

Many advancements in Machine Learning have come from using Deep Neural Networks (models with lots of layers and parameters) to learn the relevant features of and transform inputted data in novel ways. By leveraging techniques from Calculus, we can compute the gradient vector of a Neural Network's cost function. Each index of the gradient vector contains the partial derivative of the cost function with respect to a parameter, which describes the impact that each individual parameter has on the total loss of the network. The sum of all the loss that comes from every input parameter is the total loss of the network, thus making computing the gradient a key factor in minimizing the loss function.

Once the gradient is computed, the model's paramaters are adjusted using a parameter estimation method such as gradient descent, and the loss and parameters are continously adjusted and recomputed until the model's behavior more closely mimics the information it learned from the training data.

As model sizes keep increasing, an efficient algorithm is needed to compute the gradient. This algorithm should ideally be computationally efficient so as to minimize total training time and memory usage when training neural networks. This will ensure that models can be deployed into producition faster, and that they can be made more accuracte by being able to be trained on more data in less time.

## Introduction to backpropagation

Math explaining in detail here

## Pseudocode

## Code

In [54]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.bias = np.zeros((1, output_size))
        self.learning_rate = 0.01

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.bias
        self.output = self.sigmoid(self.z)
        return self.output

    def backward(self, d_output, layer_idx):
        d_z = d_output * self.sigmoid_derivative(self.output)
        d_weights = np.dot(self.inputs.T, d_z)
        d_bias = np.sum(d_z, axis=0, keepdims=True)
        d_inputs = np.dot(d_z, self.weights.T)

        equation = f"sigmoid_derivative(output) * d_output * dloss/dw_{layer_idx}"
        gradients = f"dloss/dw_{layer_idx}: {d_weights}, dloss/db_{layer_idx}: {d_bias}"

        print(f"Layer {layer_idx} - Loss wrt Layer: w_{layer_idx}, Equation of derivative: ({equation}), gradients used from previous layers: [{gradients}]")

        self.weights -= self.learning_rate * d_weights
        self.bias -= self.learning_rate * d_bias

        return d_inputs

class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            self.layers.append(Layer(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, inputs):
        print(f"Input to network: {inputs}")
        for idx, layer in enumerate(self.layers):
            inputs = layer.forward(inputs)
            print(f"Layer {idx} output: {inputs}")
        self.output = inputs
        print(f"Final output: {self.output}")
        return self.output

    def backward(self, target):
        error = target - self.output
        d_output = error
        print(f"Output error: {error}")
        for idx, layer in enumerate(reversed(self.layers)):
            d_output = layer.backward(d_output, len(self.layers) - 1 - idx)
            print(f"Error propagated to Layer {len(self.layers) - 2 - idx}: {d_output}")

    def train(self, inputs, target):
        self.forward(inputs)
        self.backward(target)

# Example usage:
if __name__ == "__main__":
    # Define the network with 2 inputs, three hidden layers with 4 neurons each, and 1 output
    nn = NeuralNetwork(layer_sizes=[2, 4, 4, 4, 1])
    
    # Training data: XOR problem
    inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    targets = np.array([[0], [1], [1], [0]])
    
    # Training the neural network
    epochs = 1
    for epoch in range(epochs):
        for i in range(len(inputs)):
            nn.train(inputs[i:i+1], targets[i:i+1])
    
    # Testing the neural network
    for i in range(len(inputs)):
        output = nn.forward(inputs[i:i+1])
        print(f"Input: {inputs[i]}, Predicted Output: {output}, Target: {targets[i]}")

Input to network: [[0 0]]
Layer 0 output: [[0.5 0.5 0.5 0.5]]
Layer 1 output: [[0.50074883 0.49790639 0.49832571 0.50156196]]
Layer 2 output: [[0.49908172 0.50361903 0.4959061  0.50294069]]
Layer 3 output: [[0.50112136]]
Final output: [[0.50112136]]
Output error: [[-0.50112136]]
Layer 3 - Loss wrt Layer: w_3, Equation of derivative: (sigmoid_derivative(output) * d_output * dloss/dw_3), gradients used from previous layers: [dloss/dw_3: [[-0.06252481]
 [-0.06309325]
 [-0.06212697]
 [-0.06300826]], dloss/db_3: [[-0.12527971]]]
Error propagated to Layer 2: [[-0.00201579 -0.00092231  0.00124345  0.00058052]]
Layer 2 - Loss wrt Layer: w_2, Equation of derivative: (sigmoid_derivative(output) * d_output * dloss/dw_2), gradients used from previous layers: [dloss/dw_2: [[-2.52350371e-04 -1.15455731e-04  1.55653690e-04  7.26711207e-05]
 [-2.50917937e-04 -1.14800361e-04  1.54770141e-04  7.22586123e-05]
 [-2.51129248e-04 -1.14897041e-04  1.54900482e-04  7.23194652e-05]
 [-2.52760146e-04 -1.15643212

## Practical Example

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

At this step, we define transformations that will turn our image datasets into tensors that pytorch can use, and normalize them, with mean $0.5$ and standard deviation $0.5$, so that values are approximately between the range $[-1,1]$. This is optimal for the allowing the gradient descent algorithm to converge to a true local minimum, and would be very necessary if you were using input features with different scales (though we aren't here).

Next, we download training and test datasets, and apply our transformation onto them, and define our batch size. This helps improve computational efficiency by computing the gradient and loss of a batch of weights, and then adjusting the parameters based on a batch, instead of once for each image.

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


Next we'll define a Neural Network class, as a child class of PyTorch's nn.Module class. In the initialization, we'll first use the initialization of the parent function, and then create our layers. Each MNIST image is a 28x28 image. After applying the transformation, pytorch flattens this automatically  into a 1D 28*28 = 784 length vector in the input layer. 

The first hidden layer fc1 thus takes 28*28 input nodes, and outputs 128, which is the input size of the next hidden layer. We then reduce the dimensionality to 64 before passing through the second activation layer. Finally, we have 10 possible outputs, representing one of the ten possible digits. 

In [3]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

        
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Next, let's create an instance of the neural net class, and then define our loss function and parameter estimation method. Cross entropy loss is a common loss function for categorical data, and stochastic gradient descent is a commonly used algorithm to update weights.

In [4]:
model = NeuralNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Now we'll train our model over 5 epochs. Notice how the loss decreases with each iteration. This is the network learning in action. The model is using the computed gradient to calculate loss, and then adjust the weights using stochastic gradient descent!

In [5]:
for epoch in range(5):  # Num of epochs
    running_loss = 0.0
    for inputs, labels in trainloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

Epoch 1, Loss: 0.4150677062968201
Epoch 2, Loss: 0.17772377693632455
Epoch 3, Loss: 0.13109835688827007
Epoch 4, Loss: 0.10235350166828727
Epoch 5, Loss: 0.08720600267828528
Finished Training


Finally, we test our model's accuracy on the test set:

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total} %')

Accuracy on the test set: 96.58 %
